In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()

In [3]:
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train=x_train.reshape(-1,28*28)
x_test=x_test.reshape(-1,28*28)

In [8]:
import torch
from torch.utils.data import TensorDataset, DataLoader

num_workers=4
batch_size=1

train_data = TensorDataset(torch.from_numpy(x_train).float(), torch.from_numpy(y_train).long())
test_data = TensorDataset(torch.from_numpy(x_test).float(), torch.from_numpy(y_test).long())


train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size,num_workers=num_workers)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size,num_workers=num_workers)


In [9]:
use_cuda=torch.cuda.is_available()
if use_cuda:
    print("Train on GPU")
else:
    print("Train on CPU")

Train on GPU


In [10]:
from torch import nn,optim
model = nn.Sequential(nn.Linear(784, 512),
                      nn.ReLU(),
                      nn.Dropout(0.2),
                      nn.Linear(512, 512),
                      nn.ReLU(),
                      nn.Dropout(0.2),
                      nn.Linear(512, 10))

# Define the loss
criterion = nn.CrossEntropyLoss()

optimizer=optim.Adam(model.parameters())


In [11]:
next(model.parameters()).is_cuda

False

In [13]:
epochs=5
for epoch in range(1,epochs+1):
    model.train()
    total_loss = 0
    for features,labels in train_loader:
        if(use_cuda):
            features,labels=features.cuda(),labels.cuda()
            
            
            optimizer.zero_grad()
            output=model(features)
            
            loss=criterion(output,labels)
            
            loss.backward()
            optimizer.step()
            
            total_loss += loss.data.item()
    print("Epoch: {}, Loss: {}".format(epoch, total_loss / len(train_loader)))
            

Epoch: 1, Loss: 0.3508641676196925
Epoch: 2, Loss: 0.3221807329546311
Epoch: 3, Loss: 0.3199403751085202
Epoch: 4, Loss: 0.3366179813961188
Epoch: 5, Loss: 0.3302399449979265


In [22]:
correct = 0
total = 0
model.eval()
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        if (use_cuda):
            images,labels=images.cuda(),labels.cuda()
            
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network is {} %'.format(100 * correct / total))

Accuracy of the network is 95.67 %
